In [1]:
import pandas as pd
import json

with open("option_columns.json","r") as f:
    option_columns = json.load(f)

#Find the dataset
# df = pd.read_csv("dataset_encoded_size.csv", dtype={k:"int8" for k in option_columns})
df = pd.read_csv("dataset_encoded_all_size.csv", dtype={k:"int8" for k in option_columns})
df.query("cid >= 30000", inplace=True)
df.fillna(-1, inplace=True)
df.query("vmlinux >= 0", inplace=True)



In [2]:
df[:10]

,X86_LOCAL_APIC,OPENVSWITCH,TEXTSEARCH_FSM,LOCKDEP_SUPPORT,GENERIC_CLOCKEVENTS_MIN_ADJUST,NETFILTER_XT_MATCH_TCPMSS,MPLS,NFC_HCI,STACKTRACE_SUPPORT,GENERIC_CMOS_UPDATE,...,XZ-bzImage,XZ-vmlinux,XZ,LZO-bzImage,LZO-vmlinux,LZO,LZ4-bzImage,LZ4-vmlinux,LZ4,cid
14758,1,0,0,1,1,0,1,0,1,1,...,5178320.0,7264848,4980068,8922064.0,11008072,8734199,9839568.0,11925896,9638560,30000
14759,1,0,0,1,1,0,0,0,1,1,...,2840016.0,4924448,2695928,4519376.0,6603288,4385061,4838864.0,6923096,4693085,30001
14760,1,0,0,1,1,0,0,0,1,1,...,8496592.0,10581024,8351248,12391888.0,14475800,12256864,13362640.0,15446872,13214970,30002
14761,1,0,0,1,1,0,0,0,1,1,...,6304720.0,8390008,6156724,8782800.0,10867576,8647251,9302992.0,11388080,9155423,30003
14762,1,0,0,1,1,0,0,1,1,1,...,12321744.0,14407032,12176312,17933264.0,20018040,17796721,19346384.0,21431472,19197696,30004
14763,1,0,1,1,1,0,0,0,1,1,...,5161936.0,7247224,5017068,8328144.0,10412920,8193340,8897488.0,10982576,8748171,30005
14764,1,0,0,1,1,0,1,0,1,1,...,6874064.0,8959352,6726164,9491408.0,11576184,9354542,10048464.0,12133552,9901074,30006
14765,1,0,0,1,1,0,0,1,1,1,...,22009616.0,24096560,21843248,33187600.0,35274032,33029312,35424016.0,37510768,35253064,30007
14766,1,0,0,1,1,0,0,0,1,1,...,3953616.0,6038904,3807028,6325200.0,8409976,6188651,6734800.0,8819888,6587832,30008
14767,1,0,0,1,1,0,1,0,1,1,...,6419408.0,8504352,6274964,9085904.0,11170328,8952807,9663440.0,11748184,9518916,30009


In [3]:
NO_ENCODED_VALUE = 0
YES_ENCODED_VALUE = 1
M_ENCODED_VALUE = 2

def nbyes(row):
    return sum(row == YES_ENCODED_VALUE)

def nbno(row):
    return sum(row == NO_ENCODED_VALUE)

def nbmodule(row):
    return sum(row == M_ENCODED_VALUE)

# df.query("kernel_size == 7304656")
# df.query("kernel_size == 7317008").apply(nbyes, axis=1) 
df.query("vmlinux == 7317008").apply(nbyes, axis=1) 

58621    240
dtype: int64

In [4]:
df['nbyes'] = df.apply(nbyes, axis=1)
df['nbno'] = df.apply(nbno, axis=1)
df['nbmodule'] = df.apply(nbmodule, axis=1)
df['nbyesmodule'] = df['nbyes'] + df['nbmodule']

In [5]:
df.sort_values(by='vmlinux', ascending=True)[['vmlinux', 'nbno', 'nbyes', 'nbmodule', 'nbyesmodule']]

,vmlinux,nbno,nbyes,nbmodule,nbyesmodule
109097,7304656,12237,241,0,241
58621,7317008,12238,240,0,240
107213,8716424,11068,741,669,1410
99232,10486184,11131,636,711,1347
99777,10504040,11351,721,406,1127
91160,10504144,10831,847,800,1647
109537,10511608,11226,724,528,1252
80771,10524528,11163,857,458,1315
66399,10572520,10731,816,931,1747
96643,10648928,10926,835,717,1552


In [14]:
size_methods = ["vmlinux", "GZIP-bzImage", "GZIP-vmlinux", "GZIP", "BZIP2-bzImage", 
              "BZIP2-vmlinux", "BZIP2", "LZMA-bzImage", "LZMA-vmlinux", "LZMA", "XZ-bzImage", "XZ-vmlinux", "XZ", 
              "LZO-bzImage", "LZO-vmlinux", "LZO", "LZ4-bzImage", "LZ4-vmlinux", "LZ4"]

In [ ]:
from sklearn.ensemble.partial_dependence import partial_dependence, plot_partial_dependence
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=["cid"]).drop(columns=size_methods), df["vmlinux"], test_size=0.1, random_state=42)  
gbreg = GradientBoostingRegressor()
gbreg.fit(X_train, y_train)
y_pred = gbreg.predict(X_test)
dfErrors = pd.DataFrame({'Actual':y_test, 'Predicted':y_pred, "error":(y_pred - y_test).abs(), "% error":((y_pred - y_test)/y_test).abs()*100})
dfErrors["% error"].describe(), gbreg



In [15]:
from sklearn.model_selection import train_test_split
from sklearn import tree

def runML(hyperparams, with_tree=False):
    X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=["cid"]).drop(columns=size_methods), df["vmlinux"], test_size=0.1, random_state=hyperparams["random_state"])  
    reg = tree.DecisionTreeRegressor(**hyperparams)
    reg.fit(X_train, y_train)
    y_pred = reg.predict(X_test)
    dfErrors = pd.DataFrame({'Actual':y_test, 'Predicted':y_pred, "error":(y_pred - y_test).abs(), "% error":((y_pred - y_test)/y_test).abs()*100})
    if with_tree:
        return dfErrors["% error"].describe(), reg
    return dfErrors["% error"].describe()

In [16]:

hyperparams = {
    "criterion": "mse",
    "max_depth": 12,
    "min_samples_leaf": 5,
    "max_leaf_nodes": None,
    "random_state": 2
}
res, reg = runML(hyperparams, with_tree=True)

In [17]:
res

count    9248.000000
mean       12.140673
std        10.820444
min         0.002288
25%         4.576733
50%         9.623098
75%        16.680300
max       239.388144
Name: % error, dtype: float64

In [18]:
col = df.drop(columns=["cid"]).drop(columns=size_methods).columns
importanceSeries = pd.Series(reg.feature_importances_, index=col.values)
importanceSeries[importanceSeries > 0].sort_values(ascending=False)

DEBUG_INFO                        3.337956e-01
nbyes                             1.919589e-01
RANDOMIZE_BASE                    1.221631e-01
DEBUG_INFO_REDUCED                1.139380e-01
DEBUG_INFO_SPLIT                  8.446913e-02
UBSAN_SANITIZE_ALL                3.554507e-02
X86_NEED_RELOCS                   2.430806e-02
KASAN                             9.840453e-03
KASAN_OUTLINE                     9.096783e-03
UBSAN_ALIGNMENT                   8.564970e-03
GCOV_PROFILE_ALL                  8.107862e-03
nbyesmodule                       4.413585e-03
XFS_DEBUG                         3.351298e-03
MISDN_W6692                       1.833235e-03
DRM_RADEON                        1.811228e-03
BLK_MQ_PCI                        1.783007e-03
DRM_NOUVEAU                       1.703177e-03
nbno                              1.574079e-03
DRM_QXL                           1.467395e-03
MLX4_EN                           1.389203e-03
KCOV_INSTRUMENT_ALL               1.102327e-03
FB_SMSCUFX   

In [19]:
importanceSeries[importanceSeries > 0].sort_values(ascending=False).to_csv("feature_importance.csv", header=True)

In [12]:
from sklearn.model_selection import train_test_split
from sklearn import ensemble

def runML_RF(hyperparams, with_tree=False):
    X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=["cid"]).drop(columns=size_methods), df["vmlinux"], test_size=0.1)  
    reg = ensemble.RandomForestRegressor(**hyperparams)
    reg.fit(X_train, y_train)
    y_pred = reg.predict(X_test)
    dfErrors = pd.DataFrame({'Actual':y_test, 'Predicted':y_pred, "error":(y_pred - y_test).abs(), "% error":((y_pred - y_test)/y_test).abs()*100})
    if with_tree:
        return dfErrors["% error"].describe(), reg
    return dfErrors["% error"].describe()


In [13]:
hyperparams = {
    "max_depth": 18,
    "n_estimators": 50,    
}
resrf, regrf = runML_RF(hyperparams, with_tree=True)
resrf

KeyboardInterrupt: 

In [ ]:
importanceRF = pd.Series(regrf.feature_importances_, index=col.values)
importanceRF[importanceRF > 0].sort_values(ascending=False)